# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment, Webservice
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import AciWebservice

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import joblib 
import requests
from pprint import pprint

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

#### Dataset description
The dataset contains data about 1,470 employees - their demographic, personal and professional characteristics, as well as information about employee attrition. "Attrition" is the target column. 

The dataset is retrieved from https://www.kaggle.com/alhassanabdelglil/classification 

To avoid access issues, I have uploaded an unpacked copy of the dataset into own Github account, and load the dataset to AzureML from this copy.

#### Task
The purpose of the model is to assess whether an employee is experiencing attrition or not, based on employee's personal and professional characteristics. This task is a classification task.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'og-azureml-capstone'
experiment=Experiment(ws, experiment_name)
run = experiment.start_logging()

In [ ]:
# Create or retrieve a compute cluster
cluster_name = list(ws.compute_targets.keys())[0]
if not(cluster_name):
    cluster_name = 'capstone-cluster'
    try:
        cluster = ComputeTarget(workspace = ws, name = cluster_name)
        print("Cluster already exists, start using it")
    except ComputeTargetException:
        compute_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", max_nodes = 4)
        cluster = ComputeTarget.create(ws, cluster_name, compute_config)

    cluster.wait_for_completion(show_output = True)
else:
    cluster = ComputeInstance(workspace = ws, name = cluster_name)
    print("Cluster already exists:", cluster_name)

In [ ]:
# Upload dataset to the workspace
data_path = "https://raw.githubusercontent.com/olgagnatenko13/nd00333-capstone/master/dataset/Dataset_for_Classification.csv"
found = False
key = "attrition-analysis"
description_text = "Employee attrition data"

if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    # Create AML Dataset and register it into Workspace
    dataset = Dataset.Tabular.from_delimited_files(data_path)        
      
    dataset = dataset.register(workspace=ws, name=key, description=description_text)

df = dataset.to_pandas_dataframe()

In [ ]:
# Split the dataset into test and train 
test_size = 0.25
random_state = 7

ready_ds = pd.concat([x, y], axis=1, sort = False)
ds_train, ds_test = train_test_split(ready_ds, test_size = test_size, random_state = random_state)

In [ ]:
# Save split datasets 
data_folder='./data'
os.makedirs(data_folder, exist_ok=True)
data_path_train = os.path.join(data_folder, "ds_train.csv")
data_path_test = os.path.join(data_folder, "ds_test.csv")

ds_train.to_csv(data_path_train, index = False)
ds_test.to_csv(data_path_test, index = False)

In [ ]:
# load split datasets
datastore = ws.get_default_datastore()
datastore.upload(src_dir = data_folder, target_path = "data", overwrite=True, show_progress=True)
tabular_ds_train = TabularDatasetFactory.from_delimited_files(path = datastore.path("data/ds_train.csv"))
tabular_ds_test = TabularDatasetFactory.from_delimited_files(path = datastore.path("data/ds_test.csv"))

## AutoML Configuration

TODO: Explain why you chose the automl settings and configuration you used below.

- experiment_timeout_minutes=20: to avoid virtual machine timeout, we limit AutoML experiment time 
- task = "classification": the selected question is a binary classification task 
- primary metric - accuracy, as in hyperparameter experiment, for compatibility purposes, and due to the fact that correct identification of positives and negatives is equally important 
- training_data = tabular_ds_train: we train the model on the "train" part of the dataset
- label_column_name = "y": the data about employee attrition are located in column "y"
- n_cross_validations = 3: we specifically indicate to use cross-validation approach instead of default train-test split, and select three cross-validation folds
- debug_log = 'automl_errors.log': place automl logs into a file for future analysis 
- compute_target = cluster_name: indicate the compute to run the experiment 

In [ ]:
# TODO: Put your automl settings here

automl_settings = (
    experiment_timeout_minutes = 20, 
    task = "classification",
    primary_metric = "accuracy",
    training_data = tabular_ds_train,
    label_column_name = "y",
    n_cross_validations = 3,
    debug_log = 'automl_errors.log',
    compute_target = cluster_name
)

# TODO: Put your automl config here
automl_config = AutoMLConfig(automl_settings)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?
**Top three models**
1. 
2. 
3. 

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
# Show Run Details
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Get best model and its details
output_folder='./outputs'

best_automl_run, best_automl_model = remote_run.get_output()
run_details = best_automl_run.get_details()

model_details = {
    'RunID': [run_details['runId']],
    'Iteration': [run_details['properties']['iteration']],
    'Primary metric': [run_details['properties']['primary_metric']],
    'Score': [run_details['properties']['score']],
    'Algorithm and hyperparameters': [best_automl_model.steps]
}

print(model_details)

In [ ]:
# Print all steps of the model 
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0]+ ' - ')
        elif hasattr(step[1], '_base_learners') and hasattr(step[1], '_meta_learner'):
            print("\nMeta Learner")
            pprint(step[1]._meta_learner)
            print()
            for estimator in step[1]._base_learners:
                print_model(estimator[1], estimator[0]+ ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(best_automl_model)

In [ ]:
#TODO: Save the best model
model_file_name = 'automl_best_model.pkl'
model_full_path = os.path.join(output_folder, model_file_name)
joblib.dump(value = best_automl_model, filename = model_full_path)

In [ ]:
# Apply best model to test dataset, and compute accuracy to compare with Hyperdrive model
from sklearn.metrics import accuracy_score

y_test = TabularDatasetFactory.from_delimited_files(path = datastore.path("data/ds_test.csv"))
y_test = y_test.keep_columns("y").to_pandas_dataframe()
tabular_ds_test = tabular_ds_test.drop_columns("y").to_pandas_dataframe()
y_predict = best_automl_model.predict(tabular_ds_test)
automl_accuracy = accuracy_score(y_test, y_predict)
print("Automl accuracy", automl_accuracy)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Register model
model = Model.register(
    workspace = ws,
    model_path = model_full_path,
    model_name = 'model_full_path',
    description = 'AutoML model predicting employee attrition'
)

In [ ]:
# Create Aci config and inference config
aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1,
    memory_gb = 1,
    description = 'AutoML model predicting employee attrition'
)

inference_config = InferenceConfig(entry_script = "score.py", environment = environment)

deployed_model_name = 'attrition-model'

In [ ]:
# Deploy the model as a web service
service = Model.deploy(
    workspace = ws,
    name = deployed_model_name,
    models = [model],
    inference_config = inference_config,
    deployment_config = aci_config,
    overwrite = True
)

service.wait_for_deployment(show_output = True)
url = service.scoring_uri
print(url)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
test_employee = ds_train.iloc[0].to_json()

In [ ]:
# Send a request to the web service
headers = {'Content-Type':'application/json'}
r = requests.post(url, data = {'data': [test_employee]} , headers = headers)
print(r.status_code)
print(r.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# print logs 
service = Webservice(ws, deployed_model_name)
print(service.get_logs())

In [ ]:
# delete web service 
service.delete()